# Notebook: Cargar y validar instancia AMPL (.dat)

Este notebook implementa un parser ligero para archivos AMPL `.dat` (formato de parámetros "param ... := ... ;"), mapea los parámetros a estructuras Python y realiza validaciones básicas para el problema de ruteo / programación de muelles. 

**Objetivos:**
- Parsear el `.dat` proveído (ej: `instancia_generada.dat`).
- Construir dataclasses para nodos, camiones y parámetros del problema.
- Validar dimensiones y consistencia de parámetros (Dist, tvia, franjas, etc.).
- Proveer un resumen legible y una visualización sencilla de la red.

> Nota: los parámetros son inmutables — cualquier cambio de parámetros debe hacerse editando el archivo `.dat` original.

In [ ]:
# Section 1: Importar librerías y configurar el entorno
import re
import json
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Any
import numpy as np
import pandas as pd
import pprint

# Optional visualization packages (install if missing)
try:
    import networkx as nx
    import matplotlib.pyplot as plt
except Exception:
    nx = None
    plt = None

# Logging and display options
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Libraries loaded. Ready to parse instance.")


In [ ]:
# Section 2: Dataclasses para nodos, camiones y parámetros
from dataclasses import dataclass

@dataclass
class Client:
    id: int
    escliente: int
    esdepo: int
    escritico: int
    DemE: float = 0.0
    DemR: float = 0.0
    TS: float = 0.0
    MinDC: float = 0.0
    MaxDC: float = 24.0

@dataclass
class Truck:
    id: int
    Cap: float
    CH: float
    CF6: float
    CF12: float
    esHora: int = 1
    esF6: int = 0
    esF12: int = 0

@dataclass
class Instance:
    clients: Dict[int, Client]
    trucks: Dict[int, Truck]
    Dist: np.ndarray
    tvia: Dict[int, np.ndarray]  # mapping f -> matrix
    v: Dict[int, float]
    tinic: Dict[int, float]
    tfin: Dict[int, float]
    params: Dict[str, Any]

    def n_nodes(self):
        return len(self.clients)

print('Dataclasses defined')


In [ ]:
# Section 3: Parser para el archivo AMPL .dat
import math

def _numtok(s):
    """Convert a token to int if integer, else float."""
    if re.match(r"^-?\d+$", s):
        return int(s)
    try:
        return float(s)
    except ValueError:
        return s


def _tokenize_pairs(body: str):
    toks = [t for t in re.split(r"\s+", body.strip()) if t != ""]
    if len(toks) == 0:
        return {}
    # Scalar single value
    if len(toks) == 1:
        return _numtok(toks[0])
    # Otherwise parse as index value pairs
    if len(toks) % 2 == 0:
        d = {}
        for i in range(0, len(toks), 2):
            idx = _numtok(toks[i])
            val = _numtok(toks[i+1])
            d[idx] = val
        return d
    # Fallback: return list
    return toks


def parse_param_blocks(text: str) -> Dict[str, Any]:
    """Parse all param blocks into a dict name -> parsed content."""
    res = {}
    # Remove C-style comments and lines starting with #
    text = re.sub(r"#.*", "", text)

    # Find top-level param blocks: param NAME [optional header] := body ;
    pattern = re.compile(r"param\s+(\w+)\s*(?:([:\[])[\s\S]*?)?:=\s*([\s\S]*?)\s*;", re.IGNORECASE)
    # Simpler incremental approach: find 'param name' and the following ';'
    for m in re.finditer(r"param\s+(\w+)\b(.*?)\s*:=\s*([\s\S]*?)\s*;", text, re.IGNORECASE):
        name = m.group(1)
        header = m.group(2).strip()
        body = m.group(3).strip()
        # Check if matrix form e.g. " : 0 1 2 := ..."
        if header.startswith(":"):
            # matrix form
            res[name] = parse_matrix_param(header, body)
        elif name.lower() == 'tvia' or body.strip().startswith('[*,*,'):
            res[name] = parse_tvia(body)
        else:
            res[name] = _tokenize_pairs(body)
    return res


def parse_matrix_param(header: str, body: str) -> Dict[Tuple[int,int], float]:
    header = header.lstrip(":").strip()
    cols = [int(x) for x in re.split(r"\s+", header) if x != ""]
    mat = {}
    for line in body.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = [p for p in re.split(r"\s+", line) if p != ""]
        # First token is row index
        row = int(parts[0])
        vals = [float(x) for x in parts[1:1+len(cols)]]
        for c_idx, col in enumerate(cols):
            mat[(row, col)] = vals[c_idx]
    # Convert to numpy matrix
    n = max([i for i,_ in mat.keys()])
    m = max([j for _,j in mat.keys()])
    arr = np.zeros((n+1, m+1), dtype=float)
    for (i,j),v in mat.items():
        arr[i,j]=v
    return arr


def parse_tvia(body: str) -> Dict[int, np.ndarray]:
    # body contains one or more blocks like [*,*,f]: 0 1 2 ... := \n rows
    blocks = {}
    # find blocks
    block_pattern = re.compile(r"\[\*,\*,(\d+)\]:\s*(.*?)\s*:=\s*([\s\S]*?)(?=(\[\*,\*,\d+\]:)|$)", re.IGNORECASE)
    for bm in block_pattern.finditer(body):
        f = int(bm.group(1))
        header = bm.group(2).strip()
        block_body = bm.group(3).strip()
        cols = [int(x) for x in re.split(r"\s+", header) if x != ""]
        mat = {}
        for line in block_body.splitlines():
            line=line.strip()
            if not line:
                continue
            parts = [p for p in re.split(r"\s+", line) if p != ""]
            row = int(parts[0])
            vals = [float(x) for x in parts[1:1+len(cols)]]
            for c_idx, col in enumerate(cols):
                mat[(row, col)] = vals[c_idx]
        n = max([i for i,_ in mat.keys()])
        m = max([j for _,j in mat.keys()])
        arr = np.zeros((n+1, m+1), dtype=float)
        for (i,j),v in mat.items():
            arr[i,j]=v
        blocks[f]=arr
    return blocks


def parse_ampl_dat(path: str) -> Dict[str, Any]:
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
    return parse_param_blocks(text)

print('Parser functions defined')


In [ ]:
# Section 4: Convertir parsed params a estructuras tipadas y validar

REQUIRED_PARAMS = ['escliente','esdepo','escritico','esHora','esF6','esF12','Cap','CH','CF6','CF12','DemE','DemR','TS','MinDC','MaxDC','Dist','tvia','v','tinic','tfin','nmuelles','durH','Lc','tcarga']


def build_instance(parsed: Dict[str, Any]) -> Instance:
    missing = [p for p in REQUIRED_PARAMS if p not in parsed]
    if missing:
        raise ValueError(f"Faltan parámetros requeridos en .dat: {missing}")

    # clients: assemble for indices in parsed['escliente'] dict
    escliente = parsed['escliente'] if isinstance(parsed['escliente'], dict) else {}
    esdepo = parsed['esdepo'] if isinstance(parsed['esdepo'], dict) else {}
    escritico = parsed['escritico'] if isinstance(parsed['escritico'], dict) else {}

    all_node_ids = sorted(set(list(escliente.keys()) + list(esdepo.keys())))
    clients = {}
    for nid in all_node_ids:
        clients[nid] = Client(
            id=nid,
            escliente=int(escliente.get(nid,0)),
            esdepo=int(esdepo.get(nid,0)),
            escritico=int(escritico.get(nid,0)),
            DemE=float(parsed['DemE'].get(nid,0.0)),
            DemR=float(parsed['DemR'].get(nid,0.0)),
            TS=float(parsed['TS'].get(nid,0.0)),
            MinDC=float(parsed['MinDC'].get(nid,0.0)),
            MaxDC=float(parsed['MaxDC'].get(nid,24.0)),
        )

    # trucks
    Cap = parsed['Cap']
    CH = parsed['CH']
    CF6 = parsed['CF6']
    CF12 = parsed['CF12']
    esHora = parsed['esHora'] if isinstance(parsed['esHora'], dict) else {}
    esF6 = parsed['esF6'] if isinstance(parsed['esF6'], dict) else {}
    esF12 = parsed['esF12'] if isinstance(parsed['esF12'], dict) else {}

    truck_ids = sorted(set(list(Cap.keys())))
    trucks = {}
    for tid in truck_ids:
        trucks[tid] = Truck(
            id=tid,
            Cap=float(Cap.get(tid)),
            CH=float(CH.get(tid)),
            CF6=float(CF6.get(tid)),
            CF12=float(CF12.get(tid)),
            esHora=int(esHora.get(tid,1)),
            esF6=int(esF6.get(tid,0)),
            esF12=int(esF12.get(tid,0)),
        )

    Dist = parsed['Dist'] if isinstance(parsed['Dist'], np.ndarray) else np.array([])
    tvia = parsed['tvia'] if isinstance(parsed['tvia'], dict) else {}
    v = {int(k): float(v) for k,v in parsed['v'].items()} if isinstance(parsed['v'], dict) else {}
    tinic = {int(k): float(v) for k,v in parsed['tinic'].items()} if isinstance(parsed['tinic'], dict) else {}
    tfin = {int(k): float(v) for k,v in parsed['tfin'].items()} if isinstance(parsed['tfin'], dict) else {}

    params = {k:v for k,v in parsed.items() if k not in ['escliente','esdepo','escritico','Cap','CH','CF6','CF12','DemE','DemR','TS','MinDC','MaxDC','Dist','tvia','v','tinic','tfin']}

    inst = Instance(clients=clients, trucks=trucks, Dist=Dist, tvia=tvia, v=v, tinic=tinic, tfin=tfin, params=params)

    # Basic consistency checks
    n_nodes = len(clients)
    if Dist.shape[0] != n_nodes or Dist.shape[1] != n_nodes:
        logger.warning(f"Dist matrix shape {Dist.shape} does not match number of nodes {n_nodes}")

    if len(tvia) == 0:
        logger.warning("tvia not parsed as blocks per franja; check file format")

    return inst

print('Builder/validator ready')


In [ ]:
# Section 5: Uso de ejemplo con la ruta que indicaste

dat_path = r"C:\Users\sjaim\Downloads\instancia_generada.dat"

try:
    parsed = parse_ampl_dat(dat_path)
    print('Parsed parameters:', ', '.join(parsed.keys()))
    inst = build_instance(parsed)
    print('\nInstance summary:')
    print('Nodes:', inst.n_nodes())
    print('Trucks:', len(inst.trucks))
    print('Franjas (v):', sorted(inst.v.items()))
    print('nmuelles:', parsed.get('nmuelles'))
    # Show a sample tvia for franja 1 from node 0->1
    if isinstance(inst.tvia, dict) and 1 in inst.tvia:
        print('Sample tvia[0,1,1] =', inst.tvia[1][0,1])
    else:
        print('tvia[1] not available')
except FileNotFoundError as e:
    print('File not found locally. When you run this notebook, place the .dat at the path above or update dat_path.')
except Exception as e:
    print('Error parsing/building instance:', e)



## Section 6: Visualización y resumen rápido

- Dibuja un grafo simple con NetworkX si está disponible.
- Resumen de demandas, ventanas y criticidad por cliente.


In [ ]:
# Visualization code (if networkx available)
if nx is None:
    print('NetworkX not available; install networkx and matplotlib to enable plotting')
else:
    G = nx.DiGraph()
    for nid, c in inst.clients.items():
        G.add_node(nid, escliente=c.escliente, escritico=c.escritico, esdepo=c.esdepo)
    # Add a few sample edges for visualization (not full graph)
    nodes = list(inst.clients.keys())
    for i in range(len(nodes)-1):
        G.add_edge(nodes[i], nodes[i+1])

    colors = []
    for n in G.nodes:
        if inst.clients[n].esdepo == 1:
            colors.append('red')
        elif inst.clients[n].escritico == 1:
            colors.append('orange')
        else:
            colors.append('lightblue')

    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G, seed=1)
    nx.draw(G, pos, with_labels=True, node_color=colors, node_size=600)
    plt.title('Nodos: rojo=depot, naranja=critico, azul=cliente')
    plt.show()

# Quick tables
clients_df = pd.DataFrame([{ 'id':c.id, 'escliente':c.escliente, 'escritico':c.escritico, 'DemE':c.DemE, 'DemR':c.DemR, 'TS':c.TS, 'MinDC':c.MinDC, 'MaxDC':c.MaxDC } for c in inst.clients.values()])
print('\nClientes summary:')
print(clients_df.head(15))

print('\nDemandas resumo: min, mean, max:', clients_df['DemE'].min(), clients_df['DemE'].mean(), clients_df['DemE'].max())


## Next steps / TODOs
- Implement the single-vector encoding and genetic operators (RBX, cut-and-fill mutation, SWAP, INSERT).
- Implement route simulator and feasibility checker (capacity over time, windows, lunch, muelles availability).
- Implement muelle scheduler using weights w1..w5 and integrate with simulator.
- Add unit tests and small instance experiments; later, add MILP baseline with PuLP for comparison.

> Si quieres, continúo ahora implementando la codificación GA y los operadores en el siguiente notebook (recomiendo un notebook separado `02_ga_operators.ipynb`) y construir pruebas unitarias sobre tu instancia.
